In [1]:
# 

# Create target object and call it y

# Create X

# Split into validation and training data

# Specify Model

# Fit Model

# Make validation predictions and calculate mean absolute error

# Show a prediction result and real data.

### Importing libraries and Reading the data.

In [2]:
# Importing the libraries
import pandas as pd
import numpy as np

In [3]:
# Impoirting the dataset
house_prices_dataset_path = "./data/house-prices-advanced-regression-techniques/sample_submission.csv"
house_prices_test_path = "./data/house-prices-advanced-regression-techniques/test.csv"
house_prices_train_path = "./data/house-prices-advanced-regression-techniques/train.csv"
house_data_discription_path = "./data/house-prices-advanced-regression-techniques/data_description.txt"

house_prices = pd.read_csv(house_prices_dataset_path)
house_prices_test = pd.read_csv(house_prices_test_path)
house_prices_train = pd.read_csv(house_prices_test_path)

In [4]:
# Exporting the data (name of columns) to .xlsx file to more convenient work.
df = pd.DataFrame(house_prices_train.columns)
df.to_excel('./auxiliary_files/show_columns.xlsx', sheet_name='Column Names')

### Classes

In [5]:
class Nan_counter:
    """Counting NaN(true) and NaN(false) values"""  
    def __init__(self, data_set):
        self.data = data_set

    def is_nan(self, col_name):
        """returning number of the NaN cells. In view True and False:
        True     1352
        False     107
        Name: column_name, dtype: int64"""
        return pd.isnull(self.data[self.col_name]).value_counts()
    
    def missing_values(self, accuracy):
        """accuracy in percentage
        #.missing_values(.01) - .01 is an accuracy in percent. 
        IOW, takes all values that less then .01 percent."""
        i = self.data.isnull().sum()
        j = len(self.data)
        y = round(i/j, 2)  
        return dict(y[y <= accuracy])

In [6]:
class Nan_columns:
    """
    - Finding the column(s) in dataframe that stored need number of NaN values,
    - Create a list with those columns"""
    def __init__(self, data_set):
        self.data = data_set
        
    def min_nan_number_in_column(self, min_required_nan):
        """finding the columns that stroed NaN where each column stored less or equal then # NaN values."""
        columns_of_dataset = self.data.columns.tolist()
        best_columns = []

        for i in columns_of_dataset:
            nan_sum = self.data[f"{i}"].isna().sum()            
            if nan_sum <= min_required_nan:
                best_columns.append(i)
        
        return best_columns
    
    def bad_column(self, val):
        """Select only those columns from the dataset with too many NaN rows.
           - `val` is percentage of bad rows"""
        columns_of_dataset = self.data.columns.tolist()
        bad_columns = []

        for i in columns_of_dataset:
            nan_sum = self.data[f"{i}"].isna().sum()            
            if val <= (nan_sum / len(self.data[f"{i}"])):
                bad_columns.append(i)
        
        return bad_columns

In [7]:
class Nan_string:
    """Show the string(s) that stored NaN value(s)"""
    def __init__(self, dataframe):
        self.df = dataframe
        
    def show_nan_string_in_dataframe(self):
        """Return only those strings that contain NaN values"""
        # creating and filling a dictionary (#str: col_name)
        nan_str_col = dict([(x, self.df.columns[y]) for x, y in zip(*np.where(self.df.isna()))])
        
        # takes only keys from dictionary
        keys = list(nan_str_col.keys())

        return self.df.loc[keys]
    
    def show_nan_string_in_column(self, col_name):
        count = 0 
        missing_str = []
        for i in house_prices_test[col_name]: 
            if i != i:
                missing_str.append(house_prices_test[col_name].index[count])
            count += 1
        return missing_str
        
    def show_nan_dict(self):
        """Return only number of the string and a name of the column that string(s) storing NaN values"""
        return dict([(x, self.df.columns[y]) for x, y in zip(*np.where(self.df.isna()))])
    


In [8]:
class Missing_column:
    """Differece between two lists(dataset and custom_sample."""
    def find_miss_col(self, col_dataframe, col_custom_sample):
        diff = set(col_dataframe.columns) - set(col_custom_sample)
        return diff

### Review Data (Understanding the Data)

In [9]:
# Counting the unique values inside column.
print(house_prices_test['LandContour'].value_counts(sort=False))

HLS      70
Low      24
Lvl    1311
Bnk      54
Name: LandContour, dtype: int64


In [10]:
# reviewing all columns and find number NaN strings inside.
#.missing_values(.01) - .01 is an accuracy in percent. IOW, takes all values that less then .01 percent.
x = Nan_counter(house_prices_test).missing_values(.01)

features_01 = list(x)

print(len(features_01))
print(features_01)

64
['Id', 'MSSubClass', 'MSZoning', 'LotArea', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'GarageCars', 'GarageArea', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SaleType', 'SaleCondition']


In [11]:
# Select dataset columns which storing less or equal than # NaN strings.
features_02 = Nan_columns(house_prices_test).min_nan_number_in_column(0)
print(len(features_02))
print(features_02)

47
['Id', 'MSSubClass', 'LotArea', 'Street', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'ExterQual', 'ExterCond', 'Foundation', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SaleCondition']


In [12]:
# Select only those columns from the dataset with too many(over 30 percent) NaN rows.
bad_data = Nan_columns(house_prices_test).bad_column(.3)

print(len(bad_data))
print(bad_data)

5
['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature']


In [13]:
# finding the missing values columns from dataset to restructure them to improve the forcast accuracy.
i = Missing_column().find_miss_col(house_prices_test, features_02)
missing_columns = i

print(house_prices_test[missing_columns].isnull().sum())

MSZoning           4
Fence           1169
Exterior2nd        1
MiscFeature     1408
BsmtCond          45
GarageCond        78
BsmtUnfSF          1
Exterior1st        1
SaleType           1
MasVnrArea        15
MasVnrType        16
GarageFinish      78
KitchenQual        1
GarageYrBlt       78
TotalBsmtSF        1
GarageArea         1
GarageType        76
BsmtFinType1      42
BsmtFinSF1         1
BsmtHalfBath       2
LotFrontage      227
BsmtFullBath       2
PoolQC          1456
GarageQual        78
Alley           1352
BsmtFinType2      42
BsmtFinSF2         1
FireplaceQu      730
BsmtExposure      44
GarageCars         1
BsmtQual          44
Functional         2
Utilities          2
dtype: int64


In [14]:
# Show str_num and col_name that storing NaN values.
show_nan_string = Nan_string(house_prices_test[missing_columns])

In [15]:
# Observing those strings in DataFrame that storing NaN values. String evaluation, visually analysis.
missing_val_list = pd.DataFrame(show_nan_string.show_nan_string_in_dataframe())
missing_val_list = missing_val_list.rename_axis('ID', index='columns')
missing_val_list

,MSZoning,Fence,Exterior2nd,MiscFeature,BsmtCond,GarageCond,BsmtUnfSF,Exterior1st,SaleType,MasVnrArea,...,GarageQual,Alley,BsmtFinType2,BsmtFinSF2,FireplaceQu,BsmtExposure,GarageCars,BsmtQual,Functional,Utilities
ID,,,,,,,,,,,,,,,,,,,,,
0,RH,MnPrv,VinylSd,NaN,TA,TA,270.0,VinylSd,WD,0.0,...,TA,NaN,LwQ,144.0,NaN,No,1.0,TA,Typ,AllPub
1,RL,NaN,Wd Sdng,Gar2,TA,TA,406.0,Wd Sdng,WD,108.0,...,TA,NaN,Unf,0.0,NaN,No,1.0,TA,Typ,AllPub
2,RL,MnPrv,VinylSd,NaN,TA,TA,137.0,VinylSd,WD,0.0,...,TA,NaN,Unf,0.0,TA,No,2.0,Gd,Typ,AllPub
3,RL,NaN,VinylSd,NaN,TA,TA,324.0,VinylSd,WD,20.0,...,TA,NaN,Unf,0.0,Gd,No,2.0,TA,Typ,AllPub
4,RL,NaN,HdBoard,NaN,TA,TA,1017.0,HdBoard,WD,0.0,...,TA,NaN,Unf,0.0,NaN,No,2.0,Gd,Typ,AllPub
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,RM,NaN,CmentBd,NaN,TA,NaN,546.0,CemntBd,WD,0.0,...,NaN,NaN,Unf,0.0,NaN,No,0.0,TA,Typ,AllPub
1455,RM,NaN,CmentBd,NaN,TA,TA,294.0,CemntBd,WD,0.0,...,TA,NaN,Unf,0.0,NaN,No,1.0,TA,Typ,AllPub
1456,RL,NaN,VinylSd,NaN,TA,TA,0.0,VinylSd,WD,0.0,...,TA,NaN,Unf,0.0,TA,No,2.0,TA,Typ,AllPub


### Recognize, Fill or drop data

In [16]:
list_num_of_nan_str = Nan_string(missing_val_list).show_nan_string_in_column('MSZoning')
list_num_of_nan_str

[455, 756, 790, 1444]

In [17]:
missing_val_list.loc[454:456][['MSZoning']]

,MSZoning
ID,
454,RM
455,NaN
456,RL


In [18]:
# Remove bad columns.
missing_val_list = missing_val_list.drop(columns=bad_data)
missing_val_list

,MSZoning,Exterior2nd,BsmtCond,GarageCond,BsmtUnfSF,Exterior1st,SaleType,MasVnrArea,MasVnrType,GarageFinish,...,LotFrontage,BsmtFullBath,GarageQual,BsmtFinType2,BsmtFinSF2,BsmtExposure,GarageCars,BsmtQual,Functional,Utilities
ID,,,,,,,,,,,,,,,,,,,,,
0,RH,VinylSd,TA,TA,270.0,VinylSd,WD,0.0,None,Unf,...,80.0,0.0,TA,LwQ,144.0,No,1.0,TA,Typ,AllPub
1,RL,Wd Sdng,TA,TA,406.0,Wd Sdng,WD,108.0,BrkFace,Unf,...,81.0,0.0,TA,Unf,0.0,No,1.0,TA,Typ,AllPub
2,RL,VinylSd,TA,TA,137.0,VinylSd,WD,0.0,None,Fin,...,74.0,0.0,TA,Unf,0.0,No,2.0,Gd,Typ,AllPub
3,RL,VinylSd,TA,TA,324.0,VinylSd,WD,20.0,BrkFace,Fin,...,78.0,0.0,TA,Unf,0.0,No,2.0,TA,Typ,AllPub
4,RL,HdBoard,TA,TA,1017.0,HdBoard,WD,0.0,None,RFn,...,43.0,0.0,TA,Unf,0.0,No,2.0,Gd,Typ,AllPub
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,RM,CmentBd,TA,NaN,546.0,CemntBd,WD,0.0,None,NaN,...,21.0,0.0,NaN,Unf,0.0,No,0.0,TA,Typ,AllPub
1455,RM,CmentBd,TA,TA,294.0,CemntBd,WD,0.0,None,Unf,...,21.0,0.0,TA,Unf,0.0,No,1.0,TA,Typ,AllPub
1456,RL,VinylSd,TA,TA,0.0,VinylSd,WD,0.0,None,Unf,...,160.0,1.0,TA,Unf,0.0,No,2.0,TA,Typ,AllPub


In [19]:
missing_val_list.isnull().sum()
list(missing_val_list.columns)
missing_val_list = missing_val_list.drop(columns=[
    'GarageQual',
    'GarageCars',
    'GarageType',
    'GarageArea', 
    'GarageFinish',
    'GarageCond',
    'GarageYrBlt'])
missing_val_list

,MSZoning,Exterior2nd,BsmtCond,BsmtUnfSF,Exterior1st,SaleType,MasVnrArea,MasVnrType,KitchenQual,TotalBsmtSF,...,BsmtFinSF1,BsmtHalfBath,LotFrontage,BsmtFullBath,BsmtFinType2,BsmtFinSF2,BsmtExposure,BsmtQual,Functional,Utilities
ID,,,,,,,,,,,,,,,,,,,,,
0,RH,VinylSd,TA,270.0,VinylSd,WD,0.0,None,TA,882.0,...,468.0,0.0,80.0,0.0,LwQ,144.0,No,TA,Typ,AllPub
1,RL,Wd Sdng,TA,406.0,Wd Sdng,WD,108.0,BrkFace,Gd,1329.0,...,923.0,0.0,81.0,0.0,Unf,0.0,No,TA,Typ,AllPub
2,RL,VinylSd,TA,137.0,VinylSd,WD,0.0,None,TA,928.0,...,791.0,0.0,74.0,0.0,Unf,0.0,No,Gd,Typ,AllPub
3,RL,VinylSd,TA,324.0,VinylSd,WD,20.0,BrkFace,Gd,926.0,...,602.0,0.0,78.0,0.0,Unf,0.0,No,TA,Typ,AllPub
4,RL,HdBoard,TA,1017.0,HdBoard,WD,0.0,None,Gd,1280.0,...,263.0,0.0,43.0,0.0,Unf,0.0,No,Gd,Typ,AllPub
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,RM,CmentBd,TA,546.0,CemntBd,WD,0.0,None,TA,546.0,...,0.0,0.0,21.0,0.0,Unf,0.0,No,TA,Typ,AllPub
1455,RM,CmentBd,TA,294.0,CemntBd,WD,0.0,None,TA,546.0,...,252.0,0.0,21.0,0.0,Unf,0.0,No,TA,Typ,AllPub
1456,RL,VinylSd,TA,0.0,VinylSd,WD,0.0,None,TA,1224.0,...,1224.0,0.0,160.0,1.0,Unf,0.0,No,TA,Typ,AllPub


In [20]:
# Fill the column that storing greater than 4 missing values
issue_last = missing_val_list.isnull().sum()
issue_last = list(issue_last[issue_last>4].index)
issue_last

one_val_list = pd.DataFrame(house_prices_test[issue_last])
one_val_list = one_val_list.rename_axis('ID', index='columns')
# one_val_list = one_val_list.dropna()
one_val_list

,BsmtCond,MasVnrArea,MasVnrType,BsmtFinType1,LotFrontage,BsmtFinType2,BsmtExposure,BsmtQual
ID,,,,,,,,
0,TA,0.0,None,Rec,80.0,LwQ,No,TA
1,TA,108.0,BrkFace,ALQ,81.0,Unf,No,TA
2,TA,0.0,None,GLQ,74.0,Unf,No,Gd
3,TA,20.0,BrkFace,GLQ,78.0,Unf,No,TA
4,TA,0.0,None,ALQ,43.0,Unf,No,Gd
...,...,...,...,...,...,...,...,...
1454,TA,0.0,None,Unf,21.0,Unf,No,TA
1455,TA,0.0,None,Rec,21.0,Unf,No,TA
1456,TA,0.0,None,ALQ,160.0,Unf,No,TA


In [21]:
# Concatanation several dataframes.

# clear_data = pd.concat([one_val_list, multi_val_list], axis=1)
# clear_data

### Hand Work

In [22]:
one_val_list.isnull().sum()

BsmtCond         45
MasVnrArea       15
MasVnrType       16
BsmtFinType1     42
LotFrontage     227
BsmtFinType2     42
BsmtExposure     44
BsmtQual         44
dtype: int64

In [24]:
# one_val_list[col_name]=one_val_list[col_name].fillna('None')

# one_val_list[col_name]=one_val_list[col_name].fillna(one_val_list[col_name]).mean()

In [25]:
col_name = 'MasVnrType'
print("The unique elements:\n")
print(one_val_list[col_name].value_counts())
print("\nThe Null values:", one_val_list[col_name].isnull().sum())

The unique elements:

None       878
BrkFace    434
Stone      121
BrkCmn      10
Name: MasVnrType, dtype: int64

The Null values: 16


In [26]:
col_name = 'LotFrontage'
print("The unique elements:\n")
print(one_val_list[col_name].value_counts())
print("\nThe Null values:", one_val_list[col_name].isnull().sum())

The unique elements:

60.0     133
80.0      68
70.0      63
50.0      60
75.0      52
        ... 
22.0       1
136.0      1
149.0      1
31.0       1
131.0      1
Name: LotFrontage, Length: 115, dtype: int64

The Null values: 227
